In [14]:
import cv2, numpy as np
import sys, os
import matplotlib.pyplot as plt
import copy
from PIL import Image
import math

PI = np.pi

In [16]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [105]:
def apply_clahe (img, clipLimit=2.0, tileGridSize=(12, 12)):
    yuv = cv2.cvtColor (img, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    clahe_out = yuv.copy()
    clahe_out[..., 0] = clahe.apply (yuv[..., 0])
    clahe_out = cv2.cvtColor (clahe_out, cv2.COLOR_YUV2BGR)
    return clahe_out


In [181]:
def increase_contrast (img, val=2):
    new_img = np.array (img, dtype=np.float64)
    new_img = np.clip (new_img * val, 0, 255)
    img[...] = new_img[...]
    return img

In [118]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [132]:
 def flat_field_correction (img, kernel_size=21):
    orig_shape = img.shape
    flat = cv2.medianBlur (img, kernel_size)
    cv2.imwrite ('images/flat.jpg', flat)
    flat_mean = cv2.mean (flat)
    out = img.copy()
    for i in range (3):
        out[..., i] = np.divide (img[..., i] * flat_mean[i], flat[..., i])
    # cv2.imshow ('flat_out', out)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return out
    

In [182]:
img_name = '../../data/img_16.jpg'
original_img = cv2.imread (img_name, cv2.IMREAD_UNCHANGED)
original_img = image_resize(original_img, width=1536)
gray = cv2.cvtColor (original_img, cv2.COLOR_BGR2GRAY)
# clahe_out = apply_clahe (original_img, 2.0, (21, 21))
# cv2.imwrite ('images/clahe_on_orig.jpg', clahe_out)
flat_out = flat_field_correction (original_img, 111)
cv2.imwrite ('images/clahe_flat.jpg', flat_out)
clahe_out = increase_contrast (flat_out, 1.5)
cv2.imwrite ('images/clahe_on_flat.jpg', clahe_out)
# orig_bright = increase_brightness (original_img, 30)
# cv2.imwrite ('images/clahe_orig_bright.jpg', orig_bright)
bright = increase_contrast (original_img, 1.5)
cv2.imwrite ('images/clahe_original_bright.jpg', bright)


True